In [ ]:
import os
import json
import random
from collections import defaultdict

current_dir = os.getcwd()

data_dir = os.path.join(current_dir, "..", "data", "generated_sentences")
data_dir = os.path.abspath(data_dir)

output_dir = os.path.join(current_dir, "..", "data", "tasks")
os.makedirs(output_dir, exist_ok=True)

model_files = [f for f in os.listdir(data_dir) if f.endswith(".json")]

grouped_data = defaultdict(list)

for file in model_files:
    with open(os.path.join(data_dir, file), "r", encoding="utf-8") as f:
        data = json.load(f)

    model_name = data["model_name"]

    for generation in data["generations"]:
        level = generation["cefr_level"]
        words = generation["word_list"]
        sentences = generation["generated_sentences"]

        for word, sentence in zip(words, sentences):
            key = (level, word)
            grouped_data[key].append({
                "model": model_name,
                "sentence": sentence
            })

tasks_by_level = defaultdict(list)

labels = ["Sentence A", "Sentence B", "Sentence C", "Sentence D", "Sentence E", "Sentence F"]

# Prompt şablonu
prompt_template = """You are a professional CEFR-aligned English sentence evaluator.

Your task is to evaluate 6 example sentences that all use the target word: "{word}" at CEFR level: {level}.

Rate each sentence from 1 (poor) to 5 (excellent) for the following **four independent criteria**:

1) Word Usage — Is the given word used with the correct meaning and appropriately in context?
2) Level Appropriateness — Are the tense, structure, and syntax appropriate for the target CEFR level (A1, A2, B1, B2, C1)?
3) Grammatical Accuracy — Are the grammatical structures correct and suitable for the expected level (simple / intermediate / advanced)?
4) Naturalness — Does the sentence sound natural and align with standard usage by native English speakers?

⚠️ Important Instructions:
- **Only return numerical ratings** for each criterion.
- **Do not include any explanations, comments, or justifications.**
- Follow the exact output format below.

### Output Format:
Sentence A: <Word Usage>, <Clarity>, <Grammar>, <Naturalness>  
Sentence B: ...  
Sentence C: ...  
Sentence D: ...  
Sentence E: ...  
Sentence F: ...  

### Sentences:
{sentences_block}
"""

for (level, word), sentence_data in grouped_data.items():
    if len(sentence_data) != 6:
        print(f"Uyarı: {level} seviyesinde '{word}' için {len(sentence_data)} cümle var, atlanıyor.")
        continue

    random.shuffle(sentence_data)

    labeled_sentences = []
    mapping = {}
    sentences_block_lines = []

    for label, item in zip(labels, sentence_data):
        sentence = item["sentence"]
        labeled_sentences.append((label, sentence))
        mapping[label] = {
            "model": item["model"],
            "level": level,
            "word": word,
            "sentence": sentence
        }
        sentences_block_lines.append(f"{label}: {sentence}")

    # Gönderilecek promptu oluştur
    sentences_block = "\n".join(sentences_block_lines)
    prompt = prompt_template.format(
        word=word,
        level=level,
        sentences_block=sentences_block
    )

    # Task nesnesi
    task = {
        "task_id": f"{level}_{word}",
        "level": level,
        "word": word,
        "labeled_sentences": labeled_sentences,
        "mapping": mapping,
        "prompt": prompt
    }

    tasks_by_level[level].append(task)

# Dosyaları seviyelere göre kaydet
for level, tasks in tasks_by_level.items():
    filename = f"tasks_{level}.json"
    filepath = os.path.join(output_dir, filename)
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(tasks, f, ensure_ascii=False, indent=2)
    print(f"{len(tasks)} görev '{filepath}' dosyasına kaydedildi.")


10 görev '/home/user/Documents/Tez/Deneyler/LLM_Degerlendirme/notebooks/../data/tasks_new/tasks_A1.json' dosyasına kaydedildi.
10 görev '/home/user/Documents/Tez/Deneyler/LLM_Degerlendirme/notebooks/../data/tasks_new/tasks_A2.json' dosyasına kaydedildi.
10 görev '/home/user/Documents/Tez/Deneyler/LLM_Degerlendirme/notebooks/../data/tasks_new/tasks_B1.json' dosyasına kaydedildi.
10 görev '/home/user/Documents/Tez/Deneyler/LLM_Degerlendirme/notebooks/../data/tasks_new/tasks_B2.json' dosyasına kaydedildi.
10 görev '/home/user/Documents/Tez/Deneyler/LLM_Degerlendirme/notebooks/../data/tasks_new/tasks_C1.json' dosyasına kaydedildi.
